# Python simulation of Photon orbits near quantum corrected blackhole

The purpose of this code is to simulate the orbits of photon near blackhole and make comparison between classical blackhole and quantum corrected blackhole.

Here we import some initial package

In [16]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

Now we define some constants 

Notice that the range of impact parameter provide faster simulation time when choose start range which is bigger than 2M.

In [26]:
M = 1 # mass of the blackhole
b_range = (2.00001*M, 50*M) # range of impact parameter
b_points = 100 # number of sample chosen for impact parameter
phi_points = 100 # number of sample chosen for azimuthal angle
t = 0.1 # the quantum fluctuation coefficient
theta = 4*np.pi/9 # angle of viewing point

We need numerical calculation here, one can consider Newton-Raphson method

In [27]:
def newton_raphson(f, df, x0, tolerance=1e-7, max_iterations=100000):

    x = x0
    for i in range(max_iterations):
        fx = f(x,t,r,M,x0)
        dfx = df(x,t,r,M,x0)

        if dfx == 0:
            return None

        x_new = x - fx / dfx

        if abs(x_new - x) < tolerance:

            return x_new

        x = x_new


    return x

Defines quantum corrected components of the metric

In [19]:
def h_theta(r,M):

    return -4*M*r*(2*M/r-1/np.tanh(r/2/M))

def h_phi(r,M):

    return -4*M*r*(2*M/r-1/np.tanh(r/2/M))
def h_rr(r,M):

    return -2*(2*M/r)**2/(1-2*M/r)**2*((2*M/r)**2-1/np.tanh((r/2/M)**2))

def h_tt(r,M):

    return -2*(2*M/r)**3*((2*M/r)**2-1/np.tanh((r/2/M)**2))

#defining Jacobian in converting spherical coordinate and Cartesian coordinate
def T(phi,theta,r,t):
    return np.array([[1,0,0,0],[0,np.cos(phi)*np.sin(theta),r*np.cos(theta)*np.cos(phi),-r*np.sin(theta)*np.sin(phi)]
            ,[0,np.sin(theta)*np.sin(phi),r*np.sin(phi)*np.cos(theta),r*np.sin(theta)*np.cos(phi)],
            [0,np.cos(theta),-r*np.sin(theta),0]])

def J(phi,theta,r,t):
    return np.linalg.inv(T(phi,theta,r,t))


# Metric to calculate P
def g00(phi,theta,r,t,M):
    return -(1-2*M/r-t*h_tt(r, M))

def g11(phi,theta,r,t,M):
    return 1/(1-2*M/r)+t*h_rr(r, M)

def g22(phi,theta,r,t,M):
    return r**2+t*h_theta(r,M)

def g33(phi,theta,r,t,M):
    return r**2*np.sin(theta)**2+t*h_phi(r,M)

def g(phi,theta,r,t,M):
    return np.array([[g00(phi,theta,r,t,M),0,0,0],[0,g11(phi,theta,r,t,M),0,0],
                     [0,0,g22(phi,theta,r,t,M),0],[0,0,0,g33(phi,theta,r,t,M)]])

#Metric to calculate gamma
def z00(phi,theta,r,t,M):
    return -(1-t*h_tt(r, M))

def z11(phi,theta,r,t,M):
    return 1+t*h_rr(r, M)

def z22(phi,theta,r,t,M):
    return r**2+t*h_theta(r,M)

def z33(phi,theta,r,t,M):
    return r**2*np.sin(theta)**2+t*h_phi(r,M)

def z(phi,theta,r,t,M):
    return np.array([[z00(phi,theta,r,t,M),0,0,0],[0,z11(phi,theta,r,t,M),0,0],
                     [0,0,z22(phi,theta,r,t,M),0],[0,0,0,z33(phi,theta,r,t,M)]])

def Z1(phi,theta,r,t,M):
    return np.dot(np.transpose(J(phi,theta,r,t)),np.dot(z(phi,theta,r,t,M),J(phi,theta,r,t)))



functions to calculate u (1/r)

In [20]:
def m(phi):
    return np.array([0,np.sin(phi),np.cos(phi),0])

def Y(theta):
    return np.array([0,0,np.sin(theta),np.sin(theta)])

def gamma(phi,theta,r,t,M):
    detY = np.sqrt(np.dot(Y(theta),np.dot(Z1(phi,theta,r,t,M),np.transpose(Y(theta)))))
    detm = np.sqrt(np.dot(m(phi),np.dot(Z1(phi,theta,r,t,M),np.transpose(m(phi)))))
    angle = np.dot(m(phi),np.dot(Z1(phi,theta,r,t,M),np.transpose(Y(theta))))/detY/detm
    return np.arccos(angle)

Define Jacobian in converting spherical coordinate and Cartesian coordinate

In [21]:
#Definition of Jacobi Elliptic function
def F(xi,k):

    return scipy.special.ellipkinc(xi,k)

def K(k):

    return scipy.special.ellipk(k)

#Definition of evaluting Jacobi function

def f(p,t,r,M,b):
    return (p**2-t*h_phi(r, M))/(1-2*M/p-t*h_tt(r, M))-b**2

def df(p,t,r,M,b):
    return (2*p*(1-2*M/p-t*h_tt(r, M))+(p**2-t*h_phi(r, M))*2*M/p**2)/(1-2*M/p-t*h_tt(r, M))**2

def P(t,r,M,b):
    p = np.real(newton_raphson(f, df, b))
    return p

def Q(t,r,M,b):
    Qs = (P(t,r,M,b)-2*M)*(P(t,r,M,b)+6*M)
    if Qs >= 0:
        return np.sqrt(Qs)
    else:
        return 0

def k(t,r,M,b):
    return np.sqrt((Q(t,r,M,b)-P(t,r,M,b)+6*M)/(2*Q(t,r,M,b)))

def xi1(t,r,M,b):
    return np.arcsin(np.sqrt((Q(t,r,M,b)-P(t,r,M,b)+2*M)/(Q(t,r,M,b)-P(t,r,M,b)+6*M)))

def u(t,r,M,b,phi,theta):
    SN = scipy.special.ellipj(gamma(phi,theta,r,t,M)/2*np.sqrt(P(t,r,M,b)/Q(t,r,M,b))
            +F(xi1(t,r,M,b),k(t,r,M,b)),k(t,r,M,b))[0]
    return (-(Q(t,r,M,b)-P(t,r,M,b)+2*M)/(4*M*P(t,r,M,b))
            +((Q(t,r,M,b)-P(t,r,M,b)+6*M)/(4*M*P(t,r,M,b)))
            *SN**2)

def u1(t,r,M,b,phi,theta):
    SN = scipy.special.ellipj((gamma(phi,theta,r,t,M)-2*np.pi)/2*np.sqrt(P(t,r,M,b)/Q(t,r,M,b))
            -F(xi1(t,r,M,b),k(t,r,M,b))+2*K(k(t,r,M,b)),k(t,r,M,b))[0]
    return (-(Q(t,r,M,b)-P(t,r,M,b)+2*M)/(4*M*P(t,r,M,b))
            +((Q(t,r,M,b)-P(t,r,M,b)+6*M)/(4*M*P(t,r,M,b)))
            *SN**2)


Define the mesh function to calculate r value at each phi and b for primary and secondary orbits

In [22]:
#define mesh for the primary orbit
def mesh(b_range,phi_range,b_points,phi_points,t,r,M,theta):
    b = np.linspace(b_range[0], b_range[1], b_points)
    phi = np.linspace(phi_range[0], phi_range[1], phi_points)
    Phi, B = np.meshgrid(b,phi)
    x, y = custom_transform(Phi, B,theta)
    Z=np.empty((0,b_points))
    for i in phi:
        array = []
        for j in b:
            array = np.append(array,np.real(1/u(t,r,M,j,i,theta)))
        Z = np.vstack((Z,array))
    return x,y,Z

#define mesh for the secondary orbit
def mesh1(b_range,phi_range,b_points,phi_points,t,r,M,theta):
    b = np.linspace(b_range[0], b_range[1], b_points)
    phi = np.linspace(phi_range[0], phi_range[1], phi_points)
    Phi, B = np.meshgrid(b,phi)
    x, y = custom_transform(Phi, B,theta)
    Z=np.empty((0,b_points))
    for i in phi:
        array = []
        for j in b:
            array = np.append(array,np.real(1/u1(t,r,M,j,i,theta)))
        Z = np.vstack((Z,array))
    return x,y,Z


The mesh function here is in Cartesian coordinate, one should change into polar coodinate

In [23]:
def custom_transform(a, b, theta):
    """
    Perform the custom transformation as described in the Mathematica code.

    Parameters:
    a : ndarray
        Radial coordinate.
    b : ndarray
        Angular coordinate.

    Returns:
    x, y : ndarray
        Transformed Cartesian coordinates.
    """
    angle = theta
    cos_angle = np.cos(angle)
    sin_angle = np.sin(angle)
    sqrt_term = np.sqrt(1 - (sin_angle**2) * (np.cos(b)**2))

    x = a * (np.cos(b) * cos_angle / sqrt_term)
    y = a * (np.sin(b) / sqrt_term)

    return x, y


This is the running code

In [ ]:
if __name__ == "__main__":
    r = 3*M
    r_array = [3*M,6*M,10*M,20*M,30*M]
    phi_range = (0, 2*np.pi)
    x_01, y_01, R_01 = mesh1(b_range, phi_range, b_points, phi_points,0,r,M,theta)
    x_00, y_00, R_00 = mesh(b_range, phi_range, b_points, phi_points,0,r,M,theta)
    x_01, y_01, R_01 = mesh1(b_range, phi_range, b_points, phi_points,0,r,M,theta)
    x_10, y_10, R_10 = mesh(b_range, phi_range, b_points, phi_points,t,r_array[0],M,theta)
    x_11, y_11, R_11 = mesh1(b_range, phi_range, b_points, phi_points,t,r_array[0],M,theta)
    x_20, y_20, R_20 = mesh(b_range, phi_range, b_points, phi_points,t,r_array[1],M,theta)
    x_21, y_21, R_21 = mesh1(b_range, phi_range, b_points, phi_points,t,r_array[1],M,theta)
    x_30, y_30, R_30 = mesh(b_range, phi_range, b_points, phi_points,t,r_array[2],M,theta)
    x_31, y_31, R_31 = mesh1(b_range, phi_range, b_points, phi_points,t,r_array[2],M,theta)
    x_40, y_40, R_40 = mesh(b_range, phi_range, b_points, phi_points,t,r_array[3],M,theta)
    x_41, y_41, R_41 = mesh1(b_range, phi_range, b_points, phi_points,t,r_array[3],M,theta)
    x_50, y_50, R_50 = mesh(b_range, phi_range, b_points, phi_points,t,r_array[4],M,theta)
    x_51, y_51, R_51 = mesh1(b_range, phi_range, b_points, phi_points,t,r_array[4],M,theta)
    
    #plot the results
    fig = plt.figure(dpi=600,figsize=(8,16))
    ax = fig.add_subplot(111)
    ax.set_aspect(2)
    
    plt.contour(x_10,y_10,R_10,[3*M],colors='black')
    plt.contour(x_11,y_11,R_11,[3*M],colors='green')
    plt.contour(x_20,y_20,R_20,[6*M],colors='blue')
    plt.contour(x_21,y_21,R_21,[6*M],colors='green')
    plt.contour(x_30,y_30,R_30,[10*M],colors='blue')
    plt.contour(x_31,y_31,R_31,[10*M],colors='green')
    plt.contour(x_40,y_40,R_40,[20*M],colors='blue')
    plt.contour(x_41,y_41,R_41,[20*M],colors='green')
    plt.contour(x_50,y_50,R_50,[30*M],colors='blue')
    plt.contour(x_51,y_51,R_51,[30*M],colors='green')
    plt.contour(x_00,y_00,R_00,[3*M,6*M,10*M,20*M,30*M],colors='black',linestyles='dashed')
    
    plt.contour(x_01,y_01,R_01,[3*M,6*M,10*M,20*M,30*M],colors='red',linestyles='dashed')
 
    plt.legend()
    plt.xlabel('Width/M')
    plt.ylabel('Length/M')
    plt.title('Quantum corrected blackhole')
    plt.show()


